In [1]:
import cPickle as pickle
import string
import os

%matplotlib inline
import numpy as np
import random
import time

import logging
from logging import info

from multiprocessing import Pool as ThreadPool
import itertools

In [2]:
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) # adds a default StreamHanlder
#root.addHandler(logging.StreamHandler())

In [6]:
root_location = "/home/shalaby/data/"
exports_location = root_location + "exported_data/"
preprocessed_data_location = root_location + "preprocessed_data/extended_pv_abs_desc_claims_full_chunks/"
new_preprocessed_data_location = root_location + "preprocessed_data/extended_pv_abs_desc_claims_full_chunks_merged/"

if not os.path.exists(new_preprocessed_data_location):
    os.makedirs(new_preprocessed_data_location)
    
doc_classifications_map_file = exports_location + "doc_classification_map.pkl"
sections_file = exports_location + "sections.pkl"
classes_file = exports_location + "classes.pkl"
subclasses_file = exports_location + "subclasses.pkl"
classifications_output = exports_location + "classifications.pkl"
training_docs_list_file = exports_location + "training_docs_list.pkl"
validation_docs_list_file = exports_location + "validation_docs_list.pkl"
test_docs_list_file = exports_location + "test_docs_list.pkl"

In [4]:
%%time
doc_classification_map = pickle.load(open(doc_classifications_map_file))
sections = pickle.load(open(sections_file))
classes = pickle.load(open(classes_file))
subclasses = pickle.load(open(subclasses_file))
training_docs_list = pickle.load(open(training_docs_list_file))
validation_docs_list = pickle.load(open(validation_docs_list_file))
test_docs_list = pickle.load(open(test_docs_list_file))

CPU times: user 36.7 s, sys: 28.9 s, total: 1min 5s
Wall time: 1min 14s


In [11]:
LINES_PER_DOCUMENT = 34
BATCH_DOCS_SIZE = 10000
NEW_BATCH_DOCS_SIZE = 50000
BATCH_SIZE_LINES = LINES_PER_DOCUMENT * BATCH_DOCS_SIZE
NEW_BATCH_SIZE_LINES = LINES_PER_DOCUMENT * NEW_BATCH_DOCS_SIZE


TRAINING_PREPROCESSED_FILES_PREFIX = preprocessed_data_location + "extended_pv_training_docs_data_preprocessed-"
VALIDATION_PREPROCESSED_FILES_PREFIX = preprocessed_data_location + "validation_docs_data_preprocessed-"
TEST_PREPROCESSED_FILES_PREFIX = preprocessed_data_location + "test_docs_data_preprocessed-"


TRAINING_NEW_PREPROCESSED_FILES_PREFIX = new_preprocessed_data_location + "extended_pv_training_docs_data_preprocessed-"
VALIDATION_NEW_PREPROCESSED_FILES_PREFIX = new_preprocessed_data_location + "validation_docs_data_preprocessed-"
TEST_NEW_PREPROCESSED_FILES_PREFIX = new_preprocessed_data_location + "test_docs_data_preprocessed-"


In [ ]:
FILE_PREFIX_TO_READ = TRAINING_PREPROCESSED_FILES_PREFIX
FILE_PREFIX_TO_WRITE = TRAINING_NEW_PREPROCESSED_FILES_PREFIX

In [ ]:
curr_index = 0
curr_read_file_index = 0
curr_write_file_index = 0

read_file = None
write_file = None

def read_line():
    global read_file, curr_read_file_index
    while True:
        if read_file is None:
            info("Reading new file for batch {}".format(curr_read_file_index))
            if os.path.exists(FILE_PREFIX_TO_READ + str(curr_read_file_index)):
                read_file = open(FILE_PREFIX_TO_READ + str(curr_read_file_index), "r")
            else: 
                raise StopIteration()
        for line in read_file:
            yield line
            
        curr_read_file_index += BATCH_DOCS_SIZE
        read_file.close()
        read_file = None

def write_line(line):
    global write_file, curr_write_file_index, curr_index
    if write_file is None:
        info("Writing to a new file for batch {}".format(curr_write_file_index))
        write_file = open(FILE_PREFIX_TO_WRITE + str(curr_write_file_index), "w")
    write_file.write(line)
    curr_index += 1
    if curr_index % NEW_BATCH_SIZE_LINES == 0:
        curr_write_file_index += NEW_BATCH_DOCS_SIZE
        write_file.close()
        write_file = None
    

for line in read_line():
    write_line(line)

2017-04-05 00:39:57,471 : INFO : Reading new file for batch 0
2017-04-05 00:39:57,472 : INFO : Writing to a new file for batch 0
2017-04-05 00:40:00,669 : INFO : Reading new file for batch 10000
2017-04-05 00:41:19,586 : INFO : Reading new file for batch 20000
2017-04-05 00:42:31,064 : INFO : Reading new file for batch 30000
2017-04-05 00:43:38,942 : INFO : Reading new file for batch 40000
2017-04-05 00:44:23,691 : INFO : Reading new file for batch 50000
2017-04-05 00:44:24,414 : INFO : Writing to a new file for batch 50000
2017-04-05 00:45:10,363 : INFO : Reading new file for batch 60000
2017-04-05 00:46:36,739 : INFO : Reading new file for batch 70000
2017-04-05 00:47:47,464 : INFO : Reading new file for batch 80000
2017-04-05 00:48:36,094 : INFO : Reading new file for batch 90000
2017-04-05 00:49:38,031 : INFO : Reading new file for batch 100000
2017-04-05 00:49:38,084 : INFO : Writing to a new file for batch 100000
2017-04-05 00:50:22,543 : INFO : Reading new file for batch 110000
